In [66]:
pip install codeflare-sdk codeflare-torchx


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [67]:
from codeflare_sdk import Cluster, ClusterConfiguration, TokenAuthentication, RayJobClient, Authentication
import os
import sys

In [68]:
# Authenticate the CodeFlare SDK
# On OpenShift, you can retrieve the token by running `oc whoami -t`,
# and the server with `oc cluster-info`.
auth = TokenAuthentication(
    token = 'sha256~fQn4wapQ51HZyAGOSLxjUMHGfB-hdt7UTpPTYphK2Qk',
    server = 'https://api.rosa-wswzx.e1i3.p3.openshiftapps.com:443',
    skip_tls=False
)
auth.login()

'Logged into https://api.rosa-wswzx.e1i3.p3.openshiftapps.com:443'

In [69]:
# Configure the Ray cluster
cluster = Cluster(ClusterConfiguration(
    head_cpu_requests=4,
    head_cpu_limits=8,
    head_memory_requests=4,
    head_memory_limits=8,
    name='dd-rl',
    namespace='hello-chris-ai-3',
    num_workers=1,
    worker_cpu_requests=4,
    worker_cpu_limits=4,
    worker_memory_requests=4,
    worker_memory_limits=4,
    # Use the following parameters with NVIDIA GPUs
    #image="quay.io/cnuland/dd-kuberay-worker:latest",
    head_extended_resource_requests={'nvidia.com/gpu':1},
    worker_extended_resource_requests={'nvidia.com/gpu':1},
    write_to_file=False,
    local_queue="user-queue"
))

Yaml resources loaded for dd-rl


Output()

In [70]:
# Create the Ray cluster
cluster.up()

Ray Cluster: 'dd-rl' has successfully been created


In [71]:
cluster.wait_ready()

Waiting for requested resources to be set up...
Requested cluster is up and running!
Dashboard is ready!


In [72]:
cluster.details()

                     🚀 CodeFlare Cluster Details 🚀                     
                                                                         
 ╭─────────────────────────────────────────────────────────────────────╮ 
 │   Name                                                              │ 
 │   dd-rl                                                 Active ✅   │ 
 │                                                                     │ 
 │   URI: ray://dd-rl-head-svc.hello-chris-ai-3.svc:10001              │ 
 │                                                                     │ 
 │   ]8;id=732303;https://ray-dashboard-dd-rl-hello-chris-ai-3.apps.rosa.rosa-wswzx.e1i3.p3.openshiftapps.com\Dashboard🔗]8;;\                                                       │ 
 │                                                                     │ 
 │                       Cluster Resources                             │ 
 │   ╭── Workers ──╮  ╭───────── Worker specs(each) ─────────╮         │ 
 │   │  # Workers  │  │  Memory      CPU         GPU         │         │ 
 │   │             │  │                                      │         │ 
 │   │  1          │  │  4G~4G       4~4         1           │         │ 
 │   │             │  │                                      │         │ 
 │   ╰─────────────╯  ╰──────────────────────────────────────╯         │ 
 ╰─────────────────────────────────────────────────────────────────────╯

RayCluster(name='dd-rl', status=<RayClusterStatus.READY: 'ready'>, head_cpu_requests=4, head_cpu_limits=8, head_mem_requests='4G', head_mem_limits='8G', num_workers=1, worker_mem_requests='4G', worker_mem_limits='4G', worker_cpu_requests=4, worker_cpu_limits=4, namespace='hello-chris-ai-3', dashboard='https://ray-dashboard-dd-rl-hello-chris-ai-3.apps.rosa.rosa-wswzx.e1i3.p3.openshiftapps.com', worker_extended_resources={'nvidia.com/gpu': 1}, head_extended_resources={'nvidia.com/gpu': 1})

In [73]:
clusterDetails = cluster.details()

                     🚀 CodeFlare Cluster Details 🚀                     
                                                                         
 ╭─────────────────────────────────────────────────────────────────────╮ 
 │   Name                                                              │ 
 │   dd-rl                                                 Active ✅   │ 
 │                                                                     │ 
 │   URI: ray://dd-rl-head-svc.hello-chris-ai-3.svc:10001              │ 
 │                                                                     │ 
 │   ]8;id=45709;https://ray-dashboard-dd-rl-hello-chris-ai-3.apps.rosa.rosa-wswzx.e1i3.p3.openshiftapps.com\Dashboard🔗]8;;\                                                       │ 
 │                                                                     │ 
 │                       Cluster Resources                             │ 
 │   ╭── Workers ──╮  ╭───────── Worker specs(each) ─────────╮         │ 
 │   │  # Workers  │  │  Memory      CPU         GPU         │         │ 
 │   │             │  │                                      │         │ 
 │   │  1          │  │  4G~4G       4~4         1           │         │ 
 │   │             │  │                                      │         │ 
 │   ╰─────────────╯  ╰──────────────────────────────────────╯         │ 
 ╰─────────────────────────────────────────────────────────────────────╯

In [78]:
# Create or connect to the Ray cluster
cluster.status()
print(f"Ray Dashboard URL: {clusterDetails.dashboard}")


# Initialize RayJobClient
# The address for RayJobClient should be the Ray head service, often exposed via a route or service URL.
# CodeFlare SDK's `cluster.details()` might provide this, or `cluster.cluster_uri()` for internal access.
# For external access (like from your notebook to submit job), you might need the Ray dashboard or a client service URL.
# Let's assume client can connect to where jobs are submitted (usually Ray head/dashboard)
# If your notebook runs outside the OpenShift cluster, you might need an external route to Ray head.
# For simplicity, using cluster_uri() which is often for internal connections; adapt if needed.
# A common pattern is to use the dashboard URL for the RayJobClient if it's accessible.
try:
    # Attempt connection to Ray head, may need adjustment based on network setup
    # For jobs, it's often `address="ray://<head_service_host>:<port>"`
    # If dashboard_url is http://...:8265, client address is often the same host but port 10001 for ray client.
    # For KubeRay, the service for Ray client is often 'raycluster-kuberay-head-svc'.
    # The dashboard_url() from codeflare might be what you need directly or with minor modification.
    
    # A common approach is to use the dashboard URL if it's correctly pointing to the Ray head service endpoint.
    # However, RayJobClient expects a Ray Client server address (ray://) or HTTP proxy.
    # If your 'cluster.dashboard_url()' provides an HTTP URL for the dashboard,
    # Ray Client can connect via HTTP Proxy if enabled (Ray >= 1.13).
    # ray_client_address = "http://" + cluster.dashboard_url().split("://")[1] # Simplified, assumes standard port or proxy setup
    
    # It's safer to get the specific client connection string if available.
    # Let's assume Codeflare SDK handles this abstraction or you can derive it.
    # If running locally and port-forwarding Ray head's client port (default 10001):
    # client_address = "ray://localhost:10001"
    # When submitting from the environment that can reach the Ray head:
    #client_address = clusterDetails.dashboard # Check if this works directly or needs adjustment for RayJobClient
                                         # It may need to be like "http://<host>:<dashboard_port>"
    client_address = "https//rayclient-dd-rl-hello-chris-ai-3.apps.rosa.rosa-wswzx.e1i3.p3.openshiftapps.com"
    print(f"Attempting to connect RayJobClient to: {client_address}")
    client = cluster.job_client


except Exception as e:
    print(f"Error initializing RayJobClient: {e}")
    print("Please ensure the Ray client server is accessible from where you are running this notebook.")
    print("You might need to set up port-forwarding or use an external route to the Ray head service.")
    client = None # Ensure client is None if connection fails

    
if client:
    print("Submitting Ray job...")

    try:
        submission_id = client.submit_job(
            entrypoint="python run-ray-dd.py",
            runtime_env={
                'working_dir': './',
                "excludes": ["*.sh", "*.ipynb", "*.md"]
            },
        )
        print(f"Job submitted successfully! Submission ID: {submission_id}")
        print("You can monitor the job status using client.get_job_status(submission_id)")
        print("And logs using client.get_job_logs(submission_id)")

        # Example: Poll for job status
        # import time
        while True:
            status = client.get_job_status(submission_id)
            print(f"Job status: {status}")
            if status in [JobStatus.SUCCEEDED, JobStatus.FAILED, JobStatus.STOPPED]:
                break
            time.sleep(5)
            print(f"Final job logs:\n{client.get_job_logs(submission_id)}")

    except Exception as e:
        print(f"Error submitting job: {e}")
else:
    print("RayJobClient not initialized. Cannot submit job.")

                      🚀 CodeFlare Cluster Status 🚀                     
                                                                         
 ╭─────────────────────────────────────────────────────────────────────╮ 
 │   Name                                                              │ 
 │   dd-rl                                                 Active ✅   │ 
 │                                                                     │ 
 │   URI: ray://dd-rl-head-svc.hello-chris-ai-3.svc:10001              │ 
 │                                                                     │ 
 │   ]8;id=62648;https://ray-dashboard-dd-rl-hello-chris-ai-3.apps.rosa.rosa-wswzx.e1i3.p3.openshiftapps.com\Dashboard🔗]8;;\                                                       │ 
 │                                                                     │ 
 ╰─────────────────────────────────────────────────────────────────────╯

2025-05-16 21:53:44,371	INFO dashboard_sdk.py:338 -- Uploading package gcs://_ray_pkg_163ee063fdb862d2.zip.
2025-05-16 21:53:44,372	INFO packaging.py:575 -- Creating a file package for local module './'.


Ray Dashboard URL: https://ray-dashboard-dd-rl-hello-chris-ai-3.apps.rosa.rosa-wswzx.e1i3.p3.openshiftapps.com
Attempting to connect RayJobClient to: https//rayclient-dd-rl-hello-chris-ai-3.apps.rosa.rosa-wswzx.e1i3.p3.openshiftapps.com
Submitting Ray job...
Job submitted successfully! Submission ID: raysubmit_6TwzzKzjNSh86Sdp
You can monitor the job status using client.get_job_status(submission_id)
And logs using client.get_job_logs(submission_id)
Job status: PENDING
Error submitting job: name 'JobStatus' is not defined


In [ ]:
client.list_jobs()

In [ ]:
client.stop_job(submission_id)

In [ ]:
cluster.down()